# Web Scraping Homework - Mission to Mars

In [1]:
#import dependencies
import os
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Setup config variables to enable Splinter interaction with browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\jchan\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


<strong> Hint:</strong> Use Splinter to navigate the sites when needed and BeautifulSoup to help find and parse out the necessary data.

## NASA Mars News

Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [3]:
# Visit Nasa news url through splinter module
nasa_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(nasa_url)

In [4]:
html = browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [5]:

# Get news title and news text by searching for appropriate div class 
news_title = soup.find('li', class_='slide').find('div', class_='content_title').text
news_p = soup.find('li', class_='slide').find('div', class_='article_teaser_body').text
print(news_title)
print(news_p)

NASA's Perseverance Rover Collects Puzzle Pieces of Mars' History
The rocks it has analyzed for sample collection are helping the team better understand a past marked by volcanic activity and water.


In [7]:
# Create dictionary to store data
scrape_nasa_news={"Title":news_title, "Paragraph":news_p}
scrape_nasa_news

NameError: name 'nasa_news_title' is not defined

## JPL Mars Space Images - Featured Image

- Visit the url for JPL Featured Space Image [here](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars).
- Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
- Find the image url to the full size .jpg image. Make sure to save a complete url string for this image.

In [ ]:
# JPL URL
jpl_url_img = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    
# Go to the JPL URL
browser.visit(jpl_url_img)

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser
soup = bs(jpl_url_img)